In [1]:
from bd import *
from rs import *
from  scrapper import *
import requests

 ········


In [2]:
lsname = "asmr"
lssize = 224

**Obtener películas de una lista**

In [3]:
def get_movies(idList):      
    sql  = "SELECT id_tittlelist FROM list_tittlelist where id_list=%s "
    with conn.cursor() as cursor:
        cursor.execute( sql, (idList) )
        value = cursor.fetchall()
        if len(value) <= 0:
            return None

    movies = []
    for m in value: movies.append(m[0])
    return movies
print(get_movies("asmr1001"))

['mulholland-drive', 'the-tree-of-life-2011', 'roma-2018', 'moonlight-2016', 'whiplash-2014', 'uncut-gems', 'uncle-boonmee-who-can-recall-', 'where-the-wild-things-are', 'birdman-or-the-unexpected-vir', 'the-diving-bell-and-the-butte']


**Obtener listas que contienen una película**

In [4]:
def get_lists(idMovie):      
    sql  = "SELECT id_list FROM list_tittlelist where id_tittlelist=%s "
    with conn.cursor() as cursor:
        cursor.execute( sql, (idMovie) )
        value = cursor.fetchall()
        if len(value) <= 0:
            return None

        lists = []
        for l in value: lists.append(l[0])
    return lists

In [5]:
mov2index = dict(); all_movies2 = []
id=1001; idm=0
while existeEnListTittleList(lsname+str(id)):
    for m in get_movies(lsname+str(id)):
        if m not in mov2index:
            mov2index[m]=idm
            all_movies2.append(m)
            idm += 1
    id += 1

In [6]:
len(mov2index)

1248

In [7]:
import random
import numpy as np
def mov2vec(movies, mov2index):
    size = len (mov2index)
    vec = np.zeros(size)
    for m in movies:
        vec[mov2index[m]] = 1.0/len(movies)
    return vec
n_data = 10000
data_x = np.zeros((n_data, len(mov2index))); data_y = np.zeros((n_data, len(mov2index)));
for i in range(0,n_data):
    j = random.randint(1001,1000+lssize)
    movies = get_movies(lsname+str(j))
    sz = len(movies)
    cut = random.randint(5,10)
    random.shuffle(movies)
    l1 = movies[:cut] # first 80% of shuffled list
    #l2 = movies[cut:] # last 20% of shuffled list
    data_x[i] = mov2vec(l1, mov2index)
    data_y[i] = mov2vec(movies, mov2index)
    

In [54]:
from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense

N = len (mov2index)
def baseline_model():
    # define the keras model
    model = Sequential()
    model.add(Dense(100, input_dim=N, activation='relu'))  # "attributes" layer
    #model.add(Dense(20, activation='relu'))  # "attributes" layer
    model.add(Dense(N))
    model.compile(loss='mse', optimizer='adam', metrics=['mse'])
    return model

In [55]:
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

estimator = KerasRegressor(build_fn=baseline_model, epochs=100, batch_size=500, verbose=False)
estimator.fit(data_x, data_y)
#kfold = KFold(n_splits=10)
#results = cross_val_score(estimator, data_x, data_y, cv=kfold)
#print("Baseline: %.2f (%.2f) MSE" % (results.mean(), results.std()))

In [56]:
def recommend(F, max_listsize=10, block_list=None, t=None, verbose=True):
    data_x = mov2vec(F, mov2index)
    
    data_y = estimator.predict([[data_x]])[0]
    
    max_indexes = (-data_y).argsort()
    recommendations = set()
    for i in max_indexes:
        
        if all_movies2[i] not in F:
            if(verbose): print(data_y[i], all_movies2[i])
            recommendations.add (all_movies2[i])
            if len(recommendations) == max_listsize: break  
    return recommendations, ""

In [57]:
#M = ['little-forest','columbus-2017','the-tale-of-the-princess-kagu','in-the-mood-for-love','kikis-delivery-service','raise-the-red-lantern','the-tree-of-life-2011','paterson','blade-runner-2049','spirited-away','arrival-2016','the-assassination-of-jesse-ja']
M = ['little-forest','columbus-2017','kikis-delivery-service','the-assassination-of-jesse-ja']
recommend(M,50,verbose=True)
pass

0.050165467 portrait-of-a-lady-on-fire
0.044624515 my-neighbor-totoro
0.043425757 raise-the-red-lantern
0.0421568 into-the-wild
0.0323621 us-and-them-2018
0.032179423 blue-jay-2016
0.032126345 i-dream-in-another-language
0.031425893 wolf-children
0.028434215 arrival-2016
0.024781276 tracks-2013
0.024473554 the-illusionist-2010
0.024139224 night-is-short-walk-on-girl
0.023511218 moonlight-2016
0.02268316 under-the-skin-2013
0.022627313 lunana-a-yak-in-the-classroom
0.021989686 rent-a-cat
0.021754943 the-many-adventures-of-winnie
0.021654148 away-2019
0.020772811 blue-2002
0.020566527 lost-in-translation
0.019393824 before-sunset
0.016242163 the-farewell-2019
0.01612431 your-name
0.015846595 crouching-tiger-hidden-dragon
0.015622478 black-swan
0.014245323 three-colors-blue
0.013951423 napoleon-dynamite
0.011983858 pi
0.011927661 howls-moving-castle
0.011740699 every-day-a-good-day
0.011711991 news-from-home
0.011530368 ad-astra-2019
0.011526038 what-dreams-may-come
0.011413553 paterson
0

In [12]:
m = get_movies(lsname+"1161")
random.shuffle(m)
l1 = m[:3] # first 80% of shuffled list
l2 = m[3:] # last 20% of shuffled list

data = mov2vec(l1, mov2index)

print(recommend(l1,10))
print(l2)
#print(np.array(data))

({'the-insider', 'kikis-delivery-service', 'little-forest', 'edward-scissorhands', 'burning-2018', 'days-of-heaven', 'under-the-skin-2013', 'koyaanisqatsi', 'call-me-by-your-name', 'my-neighbor-totoro'}, '')
['blow-out', 'dazed-and-confused', 'the-general', 'the-artist', 'scream', 'its-such-a-beautiful-day', 'white-chicks', 'star-wars']


In [13]:
all_movies=[]
id=1001
while existeEnListTittleList("asmr"+str(id)):
    all_movies += get_movies("asmr"+str(id))
    id += 1

In [14]:
len(all_movies)

2967

In [15]:
import random
random.seed(1)
def validate(list_name, nsample=3, npred=5, t=0.2):
    rw_tot_success = 0
    rand_tot_success = 0
    tot_size = 0
    
    M = get_movies(list_name) #se obtienen las pelis de la lista
    

    for m in M: 
        if m in all_movies: all_movies.remove(m)
            
    if verbose: print("movies:", M)

    #se filtran las películas que solo aprecen en la lista M
    M = [x for x in M if x in all_movies]
    
    
    if len(M)>=nsample+2:
    
        for i in range(10):
            # recommendation using random walk
            source_movies = random.sample(M,nsample) #seleccionamos nsample pelis al azar
            if verbose: print("source_movies:",source_movies)

            num = np.minimum(npred,len(M)-nsample) #número de películas a recomendar
            recommendations,_ = recommend(source_movies, max_listsize=npred, block_list=list_name, t=t) 
            MM = [x for x in M if x not in source_movies]
            rw_tot_success += len(recommendations.intersection(MM))/float(num)
            tot_size += len(recommendations)
            if verbose: print("rw_recommendations:",recommendations)

            # random recommendation

            recommendations = random.sample(all_movies,npred)
            recommendations = set(recommendations)
            rand_tot_success += len(recommendations.intersection(MM))/float(num)
            if verbose: print("r_recommendations",recommendations)
        
        print(list_name +"\t"+ str(rand_tot_success/10.0) +"\t"+ str(rw_tot_success/10.0) +"\t" \
              + str(tot_size/10.0) + "\t" + str(len(M)) )

    M = get_movies(list_name) 
    for m in M: all_movies.append(m)
    if rw_tot_success!=None: return rw_tot_success/10.0
    else: return 0.0

verbose=False
print("lista \tr_success \trw_success \t#recom \t lenM")
#validate("ms2019_ls1001", t=1.0, nsample=5, npred=10)
tot_success = 0.0; count=0
for i in range(101,lssize):
    #if i>100:
    tot_success += validate("asmr"+str(1000+i), t=1.0, nsample=5, npred=10)
    count +=1
print(tot_success/count)

lista 	r_success 	rw_success 	#recom 	 lenM
asmr1101	0.02	0.29000000000000004	10.0	18
asmr1102	0.06666666666666667	1.0	10.0	8
asmr1106	0.08	0.22000000000000003	10.0	23
asmr1107	0.03333333333333333	0.7333333333333334	10.0	8
asmr1108	0.06666666666666667	0.16666666666666666	10.0	8
asmr1109	0.05	0.25	10.0	7
asmr1110	0.07142857142857142	0.5999999999999999	10.0	12
asmr1111	0.1	0.7999999999999999	10.0	10
asmr1112	0.0	0.45	10.0	7
asmr1113	0.02	0.5599999999999999	10.0	10
asmr1114	0.06666666666666667	0.9666666666666666	10.0	8
asmr1115	0.04	0.6799999999999999	10.0	10
asmr1118	0.025	0.7	10.0	9
asmr1119	0.1	0.75	10.0	9
asmr1120	0.02	0.09999999999999999	10.0	27
asmr1121	0.06000000000000001	0.23000000000000004	10.0	18
asmr1123	0.06000000000000001	0.7999999999999999	10.0	10
asmr1124	0.05	0.23000000000000004	10.0	31
asmr1125	0.0	1.0	10.0	8
asmr1126	0.1	0.4	10.0	7
asmr1127	0.09	0.39	10.0	30
asmr1128	0.02	0.12000000000000002	10.0	10
asmr1130	0.1	0.54	10.0	10
asmr1131	0.05	0.075	10.0	9
asmr1132	0.03333333